<h2 style="color:#000000; border:2px solid #6D7E86; text-align:center; border-radius:8px; background-color:rgb(199,239,0,0.5);text-transform:uppercase;padding:0.7rem">🚀Upvote if you find it useful!</h2>


## Pet Classify Hub


### Importing Essential Packages 📦🔍


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
import cv2

### Importing Images 📦📸


In [ ]:
folder_path_train = "..\data\dogs-vs-cats\\train"
current_directory = os.getcwd()
full_path_train = os.path.join(current_directory, folder_path_train)
full_path_train = os.path.normpath(full_path_train)
print(full_path_train)

folder_path_test = "..\data\dogs-vs-cats\\test"
current_directory = os.getcwd()
full_path_test = os.path.join(current_directory, folder_path_test)
full_path_test = os.path.normpath(full_path_test)
print(full_path_test)

#### Importing all images


In [ ]:
train_images_df = []
train_labels_df = []
# plt.figure(figsize=(16, 9))
for filename in os.listdir(full_path_train):
    if filename.endswith(".jpg"):
        image_path = os.path.join(full_path_train, filename)
        if len(train_images_df) % 100 == 0:
            print(len(train_images_df))
        img = cv2.imread(image_path)
        # plt.subplot(5, 6, len(images) + 1)
        # plt.imshow(img)
        if len(train_images_df) == 1000:
            break
        train_images_df.append(img)
        train_labels_df.append(filename.split(".")[0])
print("Shape of images array:", len(train_images_df))
print("Shape of labels array:", train_labels_df)
# plt.imshow(img)

#### Imported only 1000 images for train


In [ ]:
train_images_df = []
train_labels_df = []

num_dogs = 0
num_cats = 0
target_images = 2000

for filename in os.listdir(full_path_train):
    if filename.endswith(".jpg"):
        image_path = os.path.join(full_path_train, filename)
        img = cv2.imread(image_path)

        label = filename.split(".")[0]
        if label == "cat" and num_cats < target_images / 2:
            train_images_df.append(img)
            train_labels_df.append(label)
            num_cats += 1
        elif label == "dog" and num_dogs < target_images / 2:
            train_images_df.append(img)
            train_labels_df.append(label)
            num_dogs += 1

        # Break if we have enough images
        if num_cats + num_dogs == target_images:
            break

print("Number of cat images:", num_cats)
print("Number of dog images:", num_dogs)
print("Shape of images array:", len(train_images_df))
print("Shape of labels array:", len(train_labels_df))

In [ ]:
print(train_labels_df[0:10])
print(train_labels_df[990:])

In [ ]:
# plt.figure(figsize=(40, 100))
# for i, img in enumerate(train_images):
#     plt.subplot(50, 20, i + 1)
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     plt.axis('off')

# plt.show()

#### Imported only 250 images for test


In [ ]:
predication_images = []
for filename in os.listdir(full_path_test):
    if filename.endswith(".jpg"):
        image_path = os.path.join(full_path_test, filename)
        if len(predication_images) % 100 == 0:
            print(len(predication_images))
        img = cv2.imread(image_path)
        if len(predication_images) == 250:
            break
        predication_images.append(img)
print("Shape of images array:", len(predication_images))

In [ ]:
# plt.figure(figsize=(25, 50))
# for i, img in enumerate(predication_images):
#     plt.subplot(25, 10, i + 1)
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     plt.axis('off')

# plt.show()

#### sample Submission pattern


In [ ]:
sample_submission = pd.read_csv(
    '../data/dogs-vs-cats/sampleSubmission.csv').head(250)
sample_submission

### Building and Training Models 🏗️🤖


In [ ]:
train_labels_df = np.array(train_labels_df)
train_labels_df[train_labels_df == 'cat'] = 0
train_labels_df[train_labels_df == 'dog'] = 1
train_labels_df = train_labels_df.astype(np.int32)
print(train_labels_df[0:10])
print(train_labels_df[990:])

In [ ]:
train_images_df = [cv2.resize(img, (256, 256)) for img in train_images_df]
predication_images = [cv2.resize(img, (256, 256))
                      for img in predication_images]

In [ ]:
# normalize images
train_images_df = np.array(train_images_df)
train_images_df = train_images_df / 255.0
predication_images = np.array(predication_images)
predication_images = predication_images / 255.0

In [ ]:
train_images_df.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    train_images_df, train_labels_df, test_size=0.2, random_state=42)

In [ ]:
x_test.shape

In [ ]:
x_train.shape

#### Convolutional Neural Network (CNN) 🌐🔍


In [ ]:
cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3,
                        input_shape=(256, 256, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

cnn_model.summary()

In [ ]:
cnn_model.fit(x_train, y_train, epochs=5,
              batch_size=16, verbose=1, validation_split=0.15)

In [ ]:
test_loss, test_accuracy = cnn_model.evaluate(x_test, y_test, verbose=1)
print("Accuracy on test set is: {}".format(test_accuracy))

In [ ]:
cnn_predictions = cnn_model.predict(x_test)

In [ ]:
cnn_predictions = np.round(cnn_predictions).astype(int).reshape(x_test.shape[0])

cnn_predictions[:10]

In [ ]:
cm = confusion_matrix(y_test, cnn_predictions)

In [ ]:
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

#### Making Predictions 🚀🔮


In [ ]:
y_predicted_x_test_df = cnn_model.predict(predication_images)
y_predicted_x_test_df = np.round(y_predicted_x_test_df).astype(int).reshape(predication_images.shape[0])

In [ ]:
sample_submission['label'] = y_predicted_x_test_df

sample_submission.head(10)

sample_submission.to_csv('submission.csv', index=False)

print("Visualizing Predictions 📊📈")

#### Implementing Dropout for Regularization 🕳️🔄


In [ ]:
print("Implementing Dropout for Regularization 🕳️🔄")

#### Data Augmentation for Improved Generalization 🔄📸


In [ ]:
print("Data Augmentation for Improved Generalization 🔄📸")

### Conclusion 🎉🏁


#### **Score: **
